In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


pd.set_option('chained_assignment',None)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import xgboost
from xgboost import XGBClassifier, XGBRegressor
from sklearn.tree import DecisionTreeClassifier
import catboost
from catboost import CatBoostClassifier, CatBoostRegressor
import lightgbm
from lightgbm import LGBMClassifier,LGBMRegressor
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, MinMaxScaler
from sklearn.cluster import KMeans, AgglomerativeClustering 
from sklearn.tree import ExtraTreeClassifier, ExtraTreeRegressor

In [3]:
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor
from sklearn.ensemble import BaggingClassifier, BaggingRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.gaussian_process import GaussianProcessClassifier, GaussianProcessRegressor
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.mixture import GaussianMixture
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

In [4]:
import matplotlib.pyplot as plt
import math
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error as mae
from sklearn.utils import shuffle
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, RidgeCV, SGDRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from numpy import nan as na
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier

import tensorflow as tf
import xgboost
import requests
import random
random.seed(6)
from sklearn.mixture import GaussianMixture

In [5]:
import tensorflow as tf
import xgboost
import requests
import random
random.seed(6)
from sklearn.mixture import GaussianMixture
from xgboost import XGBClassifier, XGBRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import LabelEncoder, RobustScaler
le = LabelEncoder()
from sklearn.model_selection import KFold, RepeatedKFold, GridSearchCV, cross_validate, train_test_split
from scipy.stats import pearsonr
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.cross_decomposition import PLSRegression

- # Model 1

In [6]:
df=pd.read_csv('../input/students/Train.csv')
tdf=pd.read_csv('../input/students/Test.csv')
ss=pd.read_csv('../input/students/SampleSubmission.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (140,172,254,257,275,306,343,358,373,374,461,635,641,642,643) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (172,252,254,257,275,343,358,359,461,641,642,643) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
tdf[["child_id","child_age_group","child_age"]].to_csv("first_child_age.csv",index=None)

In [8]:
tdf['child_age_group'].value_counts()

50-59 months          2409
60-69 months          1267
70 Months or older       4
Name: child_age_group, dtype: int64

In [9]:
validation=0

In [10]:
target='target'

In [11]:
index_cols = ['child_id','target']
feature_cols = ['feature_'+str(i) for i in range(1,16)]

In [12]:
def combine_columns(df, columns, strategy='add'):
    # create a new DataFrame to hold the combined columns
    combined_df = pd.DataFrame()

    # loop over all combinations of columns
    for i, col1 in enumerate(columns):
        for j, col2 in enumerate(columns):
            # skip duplicate combinations and combinations with the same column
            if i >= j or col1 == col2:
                continue
            
            # combine the columns using the specified strategy
            if strategy == 'add':
                combined = df[col1] + df[col2]
            elif strategy == 'subtract':
                combined = df[col1] - df[col2]
            elif strategy == 'multiply':
                combined = df[col1] * df[col2]
            elif strategy == 'divide':
                combined = df[col1] / df[col2]
            else:
                raise ValueError(f'Invalid strategy: {strategy}')

            # add the combined column to the new DataFrame
            combined_df[f'{col1}_{col2}_{strategy}'] = combined

    # join the combined DataFrame with the original DataFrame
    return pd.concat([df, combined_df], axis=1)


def missing_columns(df, threshold):
    total = df.shape[0]
    missing = df.isna().sum()
    missing_percentage = (missing/total) * 100
    return list(missing_percentage[missing_percentage >= threshold].index)



missing_cols = missing_columns(df, 70)
df.drop(columns=missing_cols, inplace=True)
tdf.drop(columns=missing_cols, inplace=True)

date_col = ['child_date','child_dob','child_enrolment_date']

for i in date_col:
    df[i] = pd.to_datetime(df[i], errors='coerce')
    tdf[i] = pd.to_datetime(tdf[i], errors='coerce')
    
date_col = ['child_date','child_dob']
def date_creation(df, columns):
    for col in date_col:
        df[col + '_day'] = df[col].dt.day
        df[col + '_month'] = df[col].dt.month
        df[col + '_year'] = df[col].dt.year
        # df = df.drop(columns=col)
    return df
df = date_creation(df, date_col)
tdf = date_creation(tdf, date_col)

In [13]:
num_cols=tdf.describe().columns
tdf[num_cols].describe()

,data_year,child_age,child_months_enrolment,child_height,child_observe_total,child_zha,ward_best,id_enumerator,id_facility,pri_days,...,id_ward_n,id_mn_n,id_dc_n,id_prov_n,child_date_day,child_date_month,child_date_year,child_dob_day,child_dob_month,child_dob_year
count,3680.000000,3680.000000,1112.000000,3019.000000,3410.000000,2995.000000,2410.000000,3092.000000,3675.000000,2219.000000,...,2410.000000,2677.000000,2914.000000,3037.000000,2926.000000,2926.000000,2926.000000,3080.000000,3080.000000,3080.000000
mean,2020.983152,58.153888,19.697842,107.137044,7.324927,-0.325307,26.432365,7154.159444,1200.837551,5.002253,...,24.899170,340.614867,488.447152,1689.449457,16.454204,8.562201,2021.237867,15.569805,6.579545,2016.599675
std,0.920746,5.256672,13.768122,6.373249,3.262990,1.104290,28.859240,8726.876568,688.161652,0.129138,...,20.321845,487.743141,435.319267,934.190267,8.703695,3.001662,0.485145,8.768003,3.570681,0.602420
min,2019.000000,49.741272,0.000000,64.199997,0.000000,-5.964430,1.000000,9.000000,1.000000,1.000000,...,2.000000,3.000000,7.000000,0.000000,1.000000,1.000000,2020.000000,1.000000,1.000000,2015.000000
25%,2021.000000,54.000000,8.000000,103.000000,5.000000,-1.042621,7.000000,502.000000,598.000000,5.000000,...,12.000000,45.000000,157.000000,1051.000000,9.000000,9.000000,2021.000000,8.000000,4.000000,2016.000000
50%,2021.000000,58.000000,19.000000,106.900002,8.000000,-0.358993,16.000000,2681.000000,1184.000000,5.000000,...,19.000000,110.000000,367.000000,1236.000000,18.000000,10.000000,2021.000000,16.000000,6.000000,2017.000000
75%,2022.000000,62.000000,32.000000,110.900000,10.000000,0.377827,31.000000,20022.000000,1798.500000,5.000000,...,30.000000,321.000000,629.000000,1832.000000,24.000000,11.000000,2022.000000,23.000000,10.000000,2017.000000
max,2022.000000,69.683777,57.000000,156.000000,12.000000,5.634917,125.000000,20090.000000,2361.000000,7.000000,...,119.000000,1448.000000,1448.000000,3214.000000,31.000000,12.000000,2022.000000,31.000000,12.000000,2018.000000


In [14]:
tdf[target] = -999
df = df.append(tdf[df.columns])

In [15]:
def location_feature(df):
    # Reference: https://www.kaggle.com/code/flaviafelicioni/wids-2023-different-locations-train-test-solved
    scale = 14
    df.loc[:,'latitude']=round(df.latitude,scale)
    df.loc[:,'longitude']=round(df.longitude,scale)
    
    df['loc_group'] = df.groupby(['latitude', 'longitude']).ngroup()
    print(f'{df.loc_group.nunique()} unique locations')
    
    return df

df = location_feature(df)

df['child_age_group'] = df['child_age_group'].replace({'70 Months or older':'60-69 months', 'Younger than 50 months':'50-59 months'})
df['child_observe_general'] = df['child_observe_attentive']+'_'+df['child_observe_concentrated']+'_'+df['child_observe_diligent']+'_'+df['child_observe_interested']
df['location_id_enumerator_count'] = df.groupby(['prov_best','id_enumerator'])['id_enumerator'].transform('count')

1670 unique locations


In [16]:
def combine_categorical_columns(df, categorical_columns):
    # create a new DataFrame to hold the combined columns
    combined_df = pd.DataFrame()

    # loop over all combinations of columns
    for i, col1 in enumerate(categorical_columns):
        for j, col2 in enumerate(categorical_columns):
            # skip duplicate combinations and combinations with the same column
            if i >= j or col1 == col2:
                continue
            
            # combine the categorical columns with underscores
            combined = df[col1].astype(str).fillna('missing') + '_' + df[col2].astype(str).fillna('missing')

            # add the combined column to the new DataFrame
            combined_df[f'{col1}_{col2}'] = combined

    # join the combined DataFrame with the original DataFrame
    return pd.concat([df, combined_df], axis=1)

In [17]:
df['child_year_dff'] = df['child_date_year'] - df['child_dob_year']
# Create new feature for age at enrollment
df['child_enrollment_age'] = np.floor((df['child_enrolment_date'] - df['child_dob']) / np.timedelta64(1, 'M'))

# Create new feature for duration of enrollment in months
df['enrollment_duration'] = np.floor((df['child_date'] - df['child_enrolment_date']) / np.timedelta64(1, 'M'))

In [18]:
cos = ['count_staff_salary_paid',
       'count_staff_salary', 'count_staff_gender_female', 'count_staff_gender',
       'count_staff_contract', 'count_staff_time_full', 'count_staff_time',
       'count_staff_qual_nqf6_9', 'count_toilets_children']

for i in num_cols:#cos:
    df[i+'_std'] = df.groupby(['id_enumerator','prov_best'])[i].transform('std')
    df[i+'_mean'] = df.groupby(['id_enumerator','prov_best'])[i].transform('mean')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  import sys


In [19]:
cos = ['count_staff_salary_paid',
       'count_staff_salary', 'count_staff_gender_female', 'count_staff_gender',
       'count_staff_contract', 'count_staff_time_full', 'count_staff_time',
       'count_staff_qual_nqf6_9', 'count_toilets_children']

for i in cos:
    df[i+'_std_prov'] = df.groupby(['prov_best'])[i].transform('std')
    df[i+'_mean_prov'] = df.groupby(['prov_best'])[i].transform('mean')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


In [20]:
cos = ['count_staff_salary_paid',
       'count_staff_salary', 'count_staff_gender_female', 'count_staff_gender',
       'count_staff_contract', 'count_staff_time_full', 'count_staff_time',
       'count_staff_qual_nqf6_9', 'count_toilets_children']

for i in num_cols:#cos:
    df[i+'_std_enum'] = df.groupby(['id_enumerator'])[i].transform('std')
    df[i+'_mean_enum'] = df.groupby(['id_enumerator'])[i].transform('mean')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


In [21]:
df['id_enumerator'].value_counts()

1324.0     132
1300.0     122
20060.0    117
502.0      117
2796.0     115
          ... 
20018.0      1
213.0        1
132.0        1
2836.0       1
208.0        1
Name: id_enumerator, Length: 214, dtype: int64

In [22]:
df['prov_best'].value_counts()

WESTERN CAPE     3283
KWAZULU-NATAL    1526
LIMPOPO          1276
EASTERN CAPE     1210
GAUTENG          1136
NORTH WEST       1026
MPUMALANGA        941
NORTHERN CAPE     907
FREE STATE        641
UNKNOWN           319
Name: prov_best, dtype: int64

In [23]:
cos = ['count_register_race_white',
       'count_register_race_white', 'count_register_race_indian']

for i in cos:
    df[i+'_std'] = df.groupby(['id_enumerator','prov_best'])[i].transform('std')
    df[i+'_mean'] = df.groupby(['id_enumerator','prov_best'])[i].transform('mean')

In [24]:
cos = ['latitude', 'longitude']
for i in cos:
    df[i+'_mean'] = df.groupby('id_enumerator')[i].transform('mean')

df = combine_columns(df, cos, 'divide')

cos = ['count_staff_all', 'count_children_present','count_children_attendance']
for i in cos:
    df[i+'_std'] = df.groupby(['id_enumerator','prov_best'])[i].transform('std')
    df[i+'_count'] = df.groupby(['id_enumerator','prov_best'])[i].transform('count')
    df[i+'_mean'] = df.groupby(['id_enumerator','prov_best'])[i].transform('mean')
    
df = combine_columns(df, cos, 'divide')
df = combine_columns(df, cos, 'add')

In [25]:
cos = ['teacher_emotional_total', 'pri_fees_amount_pv','pri_fees_amount']
for i in cos:
    df[i+'_std'] = df.groupby(['id_enumerator','prov_best'])[i].transform('std')
    df[i+'_count'] = df.groupby(['id_enumerator','prov_best'])[i].transform('count')
    df[i+'_mean'] = df.groupby(['id_enumerator','prov_best'])[i].transform('mean')

df = combine_columns(df, cos, 'multiply')
df = combine_columns(df, cos, 'subtract')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  after removing the cwd from sys.path.


In [26]:
cos = ['child_age', 'child_height','child_observe_total']
for i in cos:
    df[i+'_mean'] = df.groupby(['id_enumerator','prov_best'])[i].transform('mean')
    df[i+'_count'] = df.groupby(['id_enumerator','prov_best'])[i].transform('count')
    df[i+'_std'] = df.groupby(['id_enumerator','prov_best'])[i].transform('std')

df = combine_columns(df, ['child_age', 'child_height'], 'multiply')
df = combine_columns(df, ['child_age', 'child_height'], 'divide')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  after removing the cwd from sys.path.


In [27]:
col_drop = ['id_ward_n','language_match','id_dc_n','child_dob_day','child_date_month','count_staff_qual_nqf4_5','count_staff_qual_skills','count_toilets_adults',
            'count_staff_gender_male','count_staff_gender_other','count_staff_salary_unpaid','count_register_race_coloured','count_register_race',
             'count_register_race_african','count_register_race_other','count_register_year_2021', 'count_register_year_2020', 'count_register_year_2019', 'count_register_year_2018',
'count_register_year_2017', 'count_register_year_2016', 'count_register_year_2015', 'count_register_year_2014',
'count_register_year_2013', 'count_register_year_school', 'count_register_year_grader',
           'count_register_gender_female', 'count_register_gender_male', 'count_register_gender_other',
'count_register_gender','count_register_all','count_children_precovid','teacher_social_total','teacher_selfcare_total','obs_classrooms',
'pri_days','pri_time_open_minutes','pri_year','pri_time_close_minutes','pri_time_open_hours','pri_time_close_hours','id_facility']

#df = df.drop(columns=col_drop)

In [28]:
cat_cols=df.columns[df.dtypes=="object"][1:]
for col in cat_cols:
    x = list(df[col].unique())
    for i in range(len(x)):
        df[col][df[col]==x[i]]=i

In [29]:
df[cat_cols]=df[cat_cols].astype("float")
df[cat_cols].describe()

,child_grant,child_years_in_programme,child_observe_attentive,child_observe_concentrated,child_observe_diligent,child_observe_interested,child_gender,child_stunted,child_age_group,id_mn_best,...,language_child,language_assessment,facility_type,sef_ind,elp_ind,gps_ind,pre_covid,quintile_used,ses_cat,child_observe_general
count,7779.000000,11231.000000,12265.000000,12265.000000,12265.000000,12265.000000,12265.000000,9918.000000,12265.000000,8832.000000,...,10079.000000,12265.000000,4854.000000,12265.000000,10079.000000,11043.000000,12265.000000,11230.000000,8104.000000,12265.000000
mean,1.892660,1.655151,1.133469,1.569588,1.166490,1.136812,0.485446,1.069570,0.345047,40.842957,...,3.217383,3.256095,1.598063,0.158907,0.665145,0.189894,0.136812,0.629475,2.157206,20.568773
std,0.402386,0.824783,0.865767,1.179256,0.993046,0.949149,0.499809,0.288951,0.475403,39.359352,...,2.370159,2.383499,0.834653,0.365605,0.471963,0.392235,0.343663,0.482967,1.106660,25.620134
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,8.000000,...,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,4.000000
50%,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,0.000000,1.000000,0.000000,28.000000,...,3.000000,3.000000,2.000000,0.000000,1.000000,0.000000,0.000000,1.000000,2.000000,9.000000
75%,2.000000,2.000000,2.000000,3.000000,2.000000,2.000000,1.000000,1.000000,1.000000,62.000000,...,4.000000,4.000000,2.000000,0.000000,1.000000,0.000000,0.000000,1.000000,3.000000,28.000000
max,4.000000,4.000000,3.000000,3.000000,3.000000,3.000000,1.000000,3.000000,1.000000,153.000000,...,12.000000,12.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000,175.000000


In [30]:
clust = df.describe().T.sort_values("count").reset_index()[['index','count']]
clust

,index,count
0,child_months_enrolment,3733.0
1,child_enrollment_age,3733.0
2,enrollment_duration,3733.0
3,pri_attendance,3817.0
4,certificate_registration_program,3847.0
...,...,...
646,prov_best,12265.0
647,sef_ind,12265.0
648,hle_ind,12265.0
649,loc_group,12265.0


In [31]:
clust['count'].value_counts().head(21)

9237.0     31
7207.0     30
8832.0     30
9209.0     30
8835.0     29
11946.0    18
12265.0    16
10254.0    16
7406.0     13
7441.0     12
7410.0     11
7196.0     11
9194.0     11
9196.0     11
7392.0     10
8807.0      9
8809.0      9
4946.0      7
7132.0      6
9660.0      6
6408.0      6
Name: count, dtype: int64

In [32]:
cols_0 = clust[clust["count"]==12265]["index"].unique()
df[cols_0].dropna()

,child_age,child_observe_general,child_observe_attentive,child_observe_concentrated,child_observe_diligent,language_assessment,child_observe_interested,target,pre_covid,child_gender,child_age_group,prov_best,sef_ind,hle_ind,loc_group,data_year
0,59.000000,0.0,0.0,0.0,0.0,0.0,0.0,51.500000,0.0,0.0,0.0,0.0,0.0,0.0,1189,2022.0
1,60.163933,1.0,0.0,1.0,0.0,1.0,1.0,55.869999,1.0,0.0,1.0,1.0,0.0,0.0,-1,2019.0
2,69.000000,2.0,1.0,2.0,0.0,2.0,1.0,47.520000,0.0,1.0,1.0,2.0,1.0,0.0,-1,2022.0
3,53.000000,3.0,2.0,3.0,0.0,3.0,1.0,58.599998,0.0,1.0,0.0,2.0,0.0,0.0,510,2021.0
4,57.000000,4.0,2.0,3.0,1.0,1.0,2.0,76.599998,0.0,0.0,0.0,1.0,0.0,0.0,867,2021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3675,67.318275,73.0,1.0,2.0,0.0,5.0,2.0,-999.000000,0.0,1.0,1.0,0.0,1.0,0.0,1306,2022.0
3676,64.000000,0.0,0.0,0.0,0.0,2.0,0.0,-999.000000,0.0,0.0,1.0,2.0,0.0,0.0,405,2021.0
3677,59.000000,4.0,2.0,3.0,1.0,2.0,2.0,-999.000000,0.0,0.0,0.0,7.0,1.0,0.0,927,2021.0
3678,52.000000,18.0,0.0,0.0,3.0,4.0,3.0,-999.000000,0.0,0.0,0.0,3.0,0.0,0.0,1408,2021.0


In [33]:
cols_1 = clust[clust["count"]==7207]["index"].unique()
df[cols_1].dropna()

,obs_materials_5,obs_materials_12,obs_materials_11,obs_materials_10,obs_materials_4,obs_materials,obs_materials_3,obs_materials_2,obs_materials_1,obs_materials_7,...,obs_toilet_3,obs_materials_15,obs_toilet,obs_toilet_2,obs_materials_18,obs_materials_17,obs_materials_19,obs_materials_97,obs_materials_0,obs_materials_9
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,...,1.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0
7,1.0,1.0,2.0,2.0,1.0,3.0,2.0,1.0,1.0,2.0,...,1.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0
9,1.0,2.0,1.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,...,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0
13,1.0,2.0,1.0,1.0,1.0,5.0,2.0,1.0,1.0,1.0,...,1.0,1.0,3.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3674,1.0,1.0,2.0,2.0,2.0,8.0,1.0,1.0,1.0,1.0,...,1.0,1.0,5.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0
3676,1.0,2.0,1.0,1.0,1.0,17.0,2.0,1.0,1.0,1.0,...,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0
3677,1.0,2.0,1.0,1.0,1.0,242.0,2.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0
3678,1.0,1.0,2.0,2.0,1.0,99.0,1.0,2.0,2.0,1.0,...,1.0,1.0,8.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0


In [34]:
cols_2 = clust[clust["count"]==7410]["index"].unique()
df[cols_2].dropna()

,count_staff_gender_female,count_staff_gender,count_staff_contract,count_staff_time_full,count_staff_time,pri_days,pri_time_open_minutes,certificate_register,pri_time_close_minutes,pri_time_open_hours,pri_time_close_hours
3,3.0,5.0,5.0,4.0,5.0,5.0,0.0,1.0,0.0,7.0,17.0
4,3.0,3.0,3.0,3.0,3.0,5.0,0.0,1.0,0.0,7.0,15.0
7,11.0,11.0,11.0,11.0,11.0,5.0,30.0,2.0,0.0,6.0,17.0
9,7.0,8.0,8.0,8.0,8.0,5.0,0.0,1.0,0.0,7.0,17.0
13,4.0,4.0,4.0,4.0,4.0,5.0,0.0,1.0,0.0,7.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...
3674,5.0,7.0,7.0,7.0,7.0,5.0,0.0,3.0,0.0,7.0,16.0
3676,5.0,5.0,5.0,5.0,5.0,5.0,0.0,1.0,0.0,7.0,17.0
3677,3.0,3.0,3.0,3.0,3.0,5.0,0.0,2.0,0.0,7.0,14.0
3678,5.0,6.0,6.0,6.0,6.0,5.0,30.0,1.0,0.0,7.0,14.0


In [35]:
cols_3 = clust[clust["count"]==8832]["index"].unique()
df[cols_3].dropna()

,count_register_race_white_std,count_register_race_std,count_staff_salary_paid_std,count_register_gender_std,count_register_race_african_std,count_register_race_coloured_std,count_register_race_indian_std,count_staff_salary_unpaid_std,count_register_race_other_std,count_staff_qual_nqf4_5_std,...,pri_days_std,pri_time_open_minutes_std,pri_time_close_hours_std,pri_time_close_minutes_std,pri_year_std,count_staff_gender_male_std,pri_time_open_hours_std,count_staff_gender_female_std,count_toilets_adults_std,count_staff_salary_std
2,0.000000,118.556720,7.137333,118.556720,129.951655,17.038656,0.000000,0.000000,0.000000,5.556199,...,0.0,14.327008,0.000000,0.000000,4.287886,1.014515,0.374634,6.638097,1.910268,7.137333
3,6.059504,42.724247,3.235520,41.592144,43.626518,29.670970,4.097545,1.124494,0.000000,1.703621,...,0.0,14.778922,0.604416,13.872188,18.086361,0.752411,0.501718,3.587365,0.583086,3.940372
4,0.000000,24.283174,2.292580,22.847560,24.283174,0.000000,0.000000,0.637072,0.000000,1.135830,...,0.0,15.038412,1.271938,13.109849,11.803352,1.047353,1.010083,1.714408,0.891447,2.195511
6,0.000000,6.386705,1.502502,6.386705,6.386705,0.000000,0.000000,0.000000,0.000000,1.139624,...,0.0,14.762267,1.271221,14.274929,9.323988,0.718139,0.419643,0.918975,1.177028,1.502502
7,0.000000,40.569656,3.167524,41.177527,40.798792,0.631103,0.412266,0.206133,0.840664,1.074059,...,0.0,14.750675,1.064352,12.709778,8.234890,0.805809,0.491689,2.691052,2.528382,3.197985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3674,0.000000,23.452625,1.510199,23.452625,23.477873,0.160128,0.000000,0.000000,0.000000,1.097168,...,0.0,14.785922,0.598017,10.160653,10.493556,0.680359,0.683328,1.447227,1.074800,1.510199
3676,0.000000,40.572355,3.498228,40.652677,13.835613,33.512600,0.000000,0.684510,0.000000,2.032665,...,0.0,13.827327,0.237180,12.604906,12.441168,0.504349,0.431548,3.500674,1.626132,3.524349
3677,2.551048,22.889991,1.985488,20.873874,10.619295,20.960277,0.350338,0.000000,0.390016,1.659001,...,0.0,13.020006,1.672820,11.556887,10.787431,0.605970,0.470192,1.744201,1.089283,1.985488
3678,0.365005,21.910941,1.938683,21.908759,21.807304,0.451978,0.000000,1.405877,0.000000,1.018892,...,0.0,14.202962,1.237688,10.157942,10.757977,0.486897,0.507004,1.320801,0.496155,1.496731


In [36]:
cols_4 = clust[clust["count"]==7441]["index"].unique()
df[cols_4].dropna()

,teacher_social_nonaggressive,teacher_social_cooperate,teacher_social_assistance,teacher_social_ideas,teacher_emotional_adjust,teacher_emotional_confidence,teacher_social_initiative,teacher_emotional_understand,teacher_emotional_appropriate,teacher_emotional_independent,teacher_social_peers,teacher_emotional_selfstarter
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0
4,1.0,1.0,2.0,2.0,1.0,1.0,3.0,1.0,2.0,2.0,2.0,1.0
7,2.0,2.0,3.0,1.0,2.0,2.0,4.0,1.0,3.0,1.0,3.0,2.0
11,3.0,3.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3670,3.0,1.0,2.0,3.0,1.0,1.0,3.0,1.0,2.0,1.0,1.0,3.0
3671,3.0,2.0,3.0,4.0,2.0,2.0,4.0,1.0,3.0,1.0,4.0,3.0
3672,3.0,2.0,2.0,3.0,3.0,2.0,4.0,2.0,3.0,3.0,2.0,2.0
3674,3.0,3.0,3.0,1.0,2.0,3.0,4.0,2.0,3.0,1.0,1.0,3.0


In [37]:
cols_5 = clust[clust["count"]==7406]["index"].unique()
df[cols_5].dropna()

,pri_language_3,pri_language,pri_language_2,pri_separate,pri_language_6,pri_language_7,pri_language_8,pri_language_9,pri_language_10,pri_language_11,count_staff_salary_paid,count_staff_salary,pri_language_5
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,5.0,1.0
4,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0
7,1.0,3.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,11.0,11.0,1.0
9,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,8.0,8.0,1.0
13,1.0,4.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3674,1.0,6.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,7.0,7.0,1.0
3676,1.0,15.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,5.0,1.0
3677,1.0,15.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,1.0
3678,1.0,6.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,0.0,6.0,1.0


In [38]:
cols_6 = clust[clust["count"]==9194]["index"].unique()
df[cols_6].dropna()

,count_register_year_grader_std_enum,count_register_year_2021_std_enum,count_register_year_2018_std_enum,count_register_year_2017_std_enum,count_register_year_2016_std_enum,count_register_year_2020_std_enum,count_register_year_school_std_enum,count_register_year_2015_std_enum,count_register_year_2013_std_enum,count_register_year_2014_std_enum,count_register_year_2019_std_enum
2,70.063505,0.000000,23.678030,25.049658,19.098850,3.292735,0.000000,26.020235,0.000000,0.000000,16.065473
3,24.932763,2.135231,10.993319,10.294874,11.501281,6.031970,0.000000,12.423743,0.000000,0.000000,7.929753
4,20.288978,1.310843,3.772792,9.504539,9.412963,3.373467,0.499122,5.693650,0.255992,0.255992,2.873677
6,16.124334,0.836066,3.859908,7.647173,9.117584,4.561676,0.221572,1.107859,0.000000,0.221572,4.095728
7,31.430549,0.430979,11.152738,16.531255,16.006496,5.822201,0.140014,7.986275,0.000000,0.140014,5.233461
...,...,...,...,...,...,...,...,...,...,...,...
3674,16.112240,0.386953,5.493149,16.578904,9.292293,5.174065,0.000000,5.618914,0.000000,0.000000,8.331158
3676,12.739606,7.064300,12.470409,13.118806,10.258910,8.862230,0.000000,3.698465,0.000000,0.000000,12.877698
3677,23.355481,0.000000,7.507930,7.971042,15.576161,0.696643,1.666331,7.340750,0.167802,1.510214,3.389474
3678,14.820876,0.478347,5.068127,5.422118,10.371501,1.938303,0.000000,3.585180,0.000000,0.000000,5.965440


In [39]:
cols_7 = clust[clust["count"]==8835]["index"].unique()
df[cols_7].dropna()

,count_register_gender_mean,count_staff_contract_mean,count_staff_time_full_mean,child_height_mean_enum,count_register_race_african_mean,count_staff_salary_unpaid_mean,count_register_race_coloured_mean,pri_year_mean,count_register_race_indian_mean,count_register_race_white_mean,...,count_staff_gender_mean,count_staff_qual_nqf6_9_mean,pri_time_close_minutes_mean,count_staff_gender_other_mean,count_staff_salary_mean,pri_days_mean,pri_time_close_hours_mean,count_staff_gender_male_mean,count_toilets_adults_mean,pri_time_open_minutes_mean
2,110.157895,8.947368,8.421053,111.009836,90.894737,0.000000,19.263158,1999.052632,0.000000,0.000000,...,8.947368,1.210526,0.000000,0.0,8.947368,5.0,17.000000,1.157895,2.263158,9.473684
3,70.261905,7.440476,6.773810,106.453398,46.166667,0.476190,16.583333,2000.666667,1.297619,1.797619,...,7.440476,0.833333,7.321429,0.0,7.440476,5.0,17.178571,0.654762,1.396825,8.571429
4,42.703297,4.989011,4.802198,105.325510,41.549451,0.164835,0.000000,2002.265060,0.000000,0.000000,...,4.967033,0.065934,7.582418,0.0,4.956044,5.0,14.780220,0.516484,1.423529,13.846154
6,44.759259,5.685185,5.685185,108.247368,44.759259,0.000000,0.000000,2004.314815,0.000000,0.000000,...,5.685185,0.111111,20.000000,0.0,5.685185,5.0,14.314815,0.888889,1.537037,11.666667
7,64.296703,6.340659,6.054945,109.214286,63.483516,0.043956,0.153846,2003.356322,0.087912,0.000000,...,6.340659,0.417582,6.923077,0.0,6.340659,5.0,15.406593,0.659341,2.988235,18.131868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3671,37.307692,2.769231,2.769231,105.286486,37.307692,0.153846,0.000000,2001.576923,0.000000,0.000000,...,2.769231,0.000000,4.615385,0.0,2.769231,5.0,14.269231,0.192308,2.411765,0.000000
3672,47.937500,5.020833,4.822917,104.265000,47.416667,0.041667,0.937500,2006.731183,0.083333,0.000000,...,5.020833,0.187500,9.062500,0.0,5.020833,5.0,15.552083,0.541667,1.325581,10.625000
3676,80.962963,8.648148,8.314815,111.228571,20.481481,0.277778,60.333333,1994.481481,0.000000,0.000000,...,8.648148,0.351852,6.944444,0.0,8.648148,5.0,16.981481,0.481481,2.185185,8.888889
3677,52.814286,5.424658,5.178082,104.749333,7.873239,0.000000,45.422535,2005.714286,0.140845,0.676056,...,5.424658,0.178082,5.342466,0.0,5.424658,5.0,14.602740,0.657534,2.030769,7.397260


In [40]:
cols_8 = clust[clust["count"]==10254]["index"].unique()
df[cols_8].dropna()

,data_year_mean_enum,child_age_mean_enum,id_facility_mean_enum,teacher_emotional_total_count,pri_fees_amount_pv_count,id_enumerator_mean_enum,pri_fees_amount_count,data_year_mean,child_age_count,child_age_mean,child_height_count,id_enumerator_mean,child_observe_total_count,id_facility_mean,location_id_enumerator_count,id_enumerator
0,2021.961538,60.884615,638.846154,0.0,0.0,20005.0,0.0,2022.000000,25.0,61.240000,0.0,20005.0,25.0,585.320000,25.0,20005.0
2,2021.636364,64.575758,983.393939,42.0,61.0,20001.0,61.0,2021.636364,66.0,64.575758,61.0,20001.0,66.0,983.393939,66.0,20001.0
3,2021.000000,55.058252,1097.893204,84.0,84.0,2689.0,84.0,2021.000000,84.0,54.952381,84.0,2689.0,84.0,1004.416667,84.0,2689.0
4,2021.000000,54.479592,1483.867347,97.0,95.0,542.0,95.0,2021.000000,98.0,54.479592,98.0,542.0,98.0,1483.867347,98.0,542.0
6,2021.222222,60.145299,982.264957,0.0,54.0,20060.0,54.0,2021.207547,106.0,60.245283,103.0,20060.0,106.0,909.547170,106.0,20060.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3675,2022.000000,64.034168,2072.000000,0.0,0.0,158.0,0.0,2022.000000,25.0,64.034168,0.0,158.0,25.0,2072.000000,25.0,158.0
3676,2020.555556,62.936508,962.301587,23.0,54.0,20012.0,54.0,2020.555556,63.0,62.936508,35.0,20012.0,63.0,962.301587,63.0,20012.0
3677,2021.000000,55.533333,927.280000,17.0,73.0,2679.0,73.0,2021.000000,75.0,55.533333,75.0,2679.0,75.0,927.280000,75.0,2679.0
3678,2021.000000,54.663158,679.947368,92.0,86.0,567.0,86.0,2021.000000,92.0,54.619565,92.0,567.0,92.0,696.413043,92.0,567.0


In [41]:
cols_9 = clust[clust["count"]==7196.0]["index"].unique()
df[cols_9].dropna()

,obs_area_1,obs_area,obs_area_2,obs_area_3,obs_area_4,obs_area_5,obs_area_6,obs_area_7,obs_area_8,obs_area_0,obs_building
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0
7,1.0,3.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0
9,2.0,4.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,3.0
13,1.0,5.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
3674,1.0,8.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,4.0
3676,1.0,14.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0
3677,1.0,89.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0
3678,1.0,126.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0


In [42]:
cols_10 = clust[clust["count"]==7982.0]["index"].unique()
df[cols_10].dropna()

,id_ward_n,ward_best
0,14.0,39.0
2,24.0,85.0
3,22.0,18.0
4,30.0,10.0
6,28.0,101.0
...,...,...
3674,56.0,9.0
3676,15.0,105.0
3677,13.0,6.0
3678,8.0,3.0


In [43]:
cols_11 = clust[clust["count"]==7406.0]["index"].unique()
df[cols_11].dropna()

,pri_language_3,pri_language,pri_language_2,pri_separate,pri_language_6,pri_language_7,pri_language_8,pri_language_9,pri_language_10,pri_language_11,count_staff_salary_paid,count_staff_salary,pri_language_5
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,5.0,1.0
4,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0
7,1.0,3.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,11.0,11.0,1.0
9,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,8.0,8.0,1.0
13,1.0,4.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3674,1.0,6.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,7.0,7.0,1.0
3676,1.0,15.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,5.0,1.0
3677,1.0,15.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,1.0
3678,1.0,6.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,0.0,6.0,1.0


In [44]:
df["clust_1"] = df[cols_1].count(axis=1)
#df["clust_1"] = (df["clust_1"]==df["clust_1"].max()).astype("int")
df["clust_1"].value_counts()

30    7207
0     5058
Name: clust_1, dtype: int64

In [45]:
df["clust_2"] = df[cols_2].count(axis=1)
#df["clust_2"] = (df["clust_2"]==df["clust_2"].max()).astype("int")
df["clust_2"].value_counts()

11    7410
0     4855
Name: clust_2, dtype: int64

In [46]:
df["clust_3"] = df[cols_3].count(axis=1)
#df["clust_3"] = (df["clust_3"]==df["clust_3"].max()).astype("int")
df["clust_3"].value_counts()

30    7269
0     1870
3     1563
27    1563
Name: clust_3, dtype: int64

In [47]:
df["clust_4"] = df[cols_4].count(axis=1)
#df["clust_4"] = (df["clust_4"]==df["clust_4"].max()).astype("int")
df["clust_4"].value_counts()

12    7441
0     4824
Name: clust_4, dtype: int64

In [48]:
df["clust_5"] = df[cols_5].count(axis=1)
#df["clust_5"] = (df["clust_5"]==df["clust_5"].max()).astype("int")
df["clust_5"].value_counts()

13    7402
0     4855
11       4
2        4
Name: clust_5, dtype: int64

In [49]:
df["clust_6"] = df[cols_6].count(axis=1)
#df["clust_6"] = (df["clust_6"]==df["clust_6"].max()).astype("int")
df["clust_6"].value_counts()

11    9194
0     3071
Name: clust_6, dtype: int64

In [50]:
df["clust_7"] = df[cols_7].count(axis=1)
#df["clust_7"] = (df["clust_7"]==df["clust_7"].max()).astype("int")
df["clust_7"].value_counts()

29    7811
0     2406
28    1024
1     1024
Name: clust_7, dtype: int64

In [51]:
df["clust_8"] = df[cols_8].count(axis=1)
#df["clust_8"] = (df["clust_8"]==df["clust_8"].max()).astype("int")
df["clust_8"].value_counts()

16    10254
0      2011
Name: clust_8, dtype: int64

In [52]:
df["clust_9"] = df[cols_9].count(axis=1)
#df["clust_9"] = (df["clust_9"]==df["clust_9"].max()).astype("int")
df["clust_9"].value_counts()

11    7175
0     5048
1       21
10      21
Name: clust_9, dtype: int64

In [53]:
df["clust_10"] = df[cols_10].count(axis=1)
#df["clust_10"] = (df["clust_10"]==df["clust_10"].max()).astype("int")
df["clust_10"].value_counts()

2    7982
0    4283
Name: clust_10, dtype: int64

In [54]:
df["clust_11"] = df[cols_11].count(axis=1)
#df["clust_11"] = (df["clust_11"]==df["clust_11"].max()).astype("int")
df["clust_11"].value_counts()

13    7402
0     4855
11       4
2        4
Name: clust_11, dtype: int64

In [55]:
clust_cols = list(df.filter(regex='clust_').columns)
clust_cols

['clust_1',
 'clust_2',
 'clust_3',
 'clust_4',
 'clust_5',
 'clust_6',
 'clust_7',
 'clust_8',
 'clust_9',
 'clust_10',
 'clust_11']

In [56]:
if validation==1:
    df=df[df[target]!=-999]
    df,tdf=train_test_split(df,test_size=0.33,random_state=42)
    ss = tdf[index_cols]
    ss[feature_cols] = np.nan
    tdf[target]=-999
    df = df.append(tdf[df.columns])

In [57]:
len(ss),len(tdf)

(3680, 3680)

for clust in clust_cols:
    vals = df[clust].unique()
    for val in vals:
        x = df[df[clust]==val]
        x =x.dropna(axis=1)
        print(clust,val)
        display(x.corr().abs()[target].sort_values(ascending=False).head(10))

In [58]:
#df =pd.get_dummies(df,columns=cat_cols)

In [59]:
#df =pd.get_dummies(df,columns=clust_cols)

In [60]:
from sklearn.decomposition import TruncatedSVD


for clust in clust_cols:
    for cat in cat_cols:
        for col in num_cols:
            df[f'{clust}_{cat}_mean_{col}'] = df.groupby([clust,cat])[col].transform("mean")


In [61]:
svd = TruncatedSVD(n_components=3, n_iter=7, random_state=42)

In [62]:
df.corr().abs()[target].sort_values(ascending=False).head(20)


target                                                 1.000000
obs_materials                                          0.092686
certificate_registration_partial                       0.037418
pri_language_6                                         0.036395
id_ward                                                0.031505
pri_language_4                                         0.031297
child_enrollment_age                                   0.030500
obs_equipment_5                                        0.029633
pra_agency_play                                        0.028508
pri_attendance                                         0.028369
id_ward_mean                                           0.028295
enrollment_duration                                    0.027120
id_ward_mean_enum                                      0.027013
child_months_enrolment                                 0.026601
teacher_emotional_total_pri_fees_amount_pv_subtract    0.026305
teacher_emotional_total_pri_fees_amount_

In [63]:
drop_cols=index_cols
cols=list(df.head(1).drop(drop_cols,axis=1).describe().columns)

In [64]:
na_val=0

In [65]:
#svd.fit(df[cols].fillna(na_val))

In [66]:
tdf = df[df[target]==-999]
df = df[df[target]!=-999]

In [67]:
model1=CatBoostRegressor(silent=True)#,learning_rate=0.01,n_estimators=800)
tdf[target]=model1.fit(df[cols],df[target]).predict(tdf[cols])
if validation==1:
    scorer = pd.merge(ss[index_cols],tdf[index_cols],on='child_id',how='left')
    acc = mse(scorer.target_x,scorer.target_y)**0.5
    print(acc)

In [68]:
import shap
expaliner = shap.TreeExplainer(model1)
shap_values = expaliner.shap_values(tdf[cols].fillna(na_val))
f = {
    'f1' : [], 'f2': [], 'f3': [], 'f4': [],'f5': [],
    'f6' : [], 'f7': [], 'f8': [], 'f9': [],'f10': [],
    'f11' : [], 'f12': [], 'f13': [], 'f14': [],'f15': []
}
for shap_value in shap_values:
    arr = np.argsort(shap_value)[::-1][:15]
    for ind, a in enumerate(arr):
        name_f = f'f{ind+1}'
        f[name_f].append(cols[a])
        
ss['child_id'] = tdf['child_id'].values
ss[target]    = tdf[target].values

ss['feature_1'] = f['f1']
ss['feature_2'] = f['f2']
ss['feature_3'] = f['f3']
ss['feature_4'] = f['f4']
ss['feature_5'] = f['f5']
ss['feature_6'] = f['f6']
ss['feature_7'] = f['f7']
ss['feature_8'] = f['f8']
ss['feature_9'] = f['f9']
ss['feature_10'] = f['f10']
ss['feature_11'] = f['f11']
ss['feature_12'] = f['f12']
ss['feature_13'] = f['f13']
ss['feature_14'] = f['f14']
ss['feature_15'] = f['f15']
ss

,child_id,target,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15
0,ID_0I0999N6S,59.009747,child_observe_total,teacher_emotional_total,child_observe_diligent,child_observe_attentive,teacher_emotional_total_pri_fees_amount_multiply,teacher_emotional_total_pri_fees_amount_pv_mul...,child_gender,child_observe_interested,child_age,teacher_emotional_met,child_dob_month,child_zha,teacher_social_total,teacher_emotional_understand,id_facility
1,ID_GQ6ONJ4FP,47.383709,child_observe_total,teacher_emotional_total,child_observe_concentrated,child_observe_diligent,teacher_emotional_total_pri_fees_amount_multiply,child_observe_attentive,child_gender,teacher_social_total,child_observe_interested,child_dob_day_mean_enum,teacher_emotional_understand,count_register_year_2018_std_enum,pra_agency_understand,teacher_emotional_total_std,teacher_emotional_met
2,ID_YZ76CVRW3,46.007358,child_observe_diligent,child_observe_attentive,child_age_mean,child_date_year_std_enum,child_observe_total,child_age_std_enum,data_year_std_enum,count_toilets_children_std_prov,teacher_social_total_mean_enum,child_observe_total_mean,pri_year_mean,child_observe_concentrated,teacher_emotional_understand,child_observe_total_mean_enum,count_toilets_children_mean_prov
3,ID_BNINCRXH8,69.196859,child_observe_total,child_observe_concentrated,child_age,child_observe_diligent,child_observe_attentive,child_gender,teacher_emotional_total,child_age_child_height_divide,child_years_in_programme,pri_funding_subsidy_mean_enum,child_age_mean,id_facility_n_std,count_register_race_std_enum,count_staff_time_mean_prov,child_age_std_enum
4,ID_1U7GDTLRI,45.441186,child_observe_total,child_observe_diligent,child_observe_attentive,pri_funding_subsidy_mean_enum,count_register_race_std_enum,pri_fees_amount_pv_std,pri_fees_amount_std_enum,child_enrollment_age,count_toilets_children_mean_prov,teacher_emotional_understand,longitude_mean,obs_classrooms_std_enum,id_ward_mean,child_observe_interested,count_register_race_indian_std
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3675,ID_LBPQ2VMQZ,50.041975,child_age,child_age_group,ses_proxy,child_observe_total,child_observe_attentive,child_age_mean,child_observe_interested,child_date_month,pri_funding_subsidy_mean_enum,id_ward_mean,child_date_month_std,teacher_social_total_mean_enum,teacher_emotional_understand,child_dob_day,child_years_in_programme
3676,ID_H2RKKMMKK,56.335369,child_age,child_age_group,child_height,child_date_month,child_observe_total_mean,ward_best,child_gender,child_zha,child_date_day_mean_enum,count_register_race_std_enum,teacher_selfcare_total_mean_enum,child_observe_total_mean_enum,count_register_year_2020,child_dob_day_std_enum,child_age_child_height_multiply
3677,ID_VY8KX7YTZ,36.955589,child_observe_total,child_observe_concentrated,child_observe_diligent,child_observe_attentive,child_age,child_gender,child_observe_general,child_observe_interested,teacher_emotional_understand,child_dob_month_mean_enum,pra_agency_understand,child_dob_day_std,teacher_selfcare_total,teacher_emotional_selfstarter,teacher_selfcare_total_mean_enum
3678,ID_EO2MYZ4M7,40.470709,language_match_std_enum,child_observe_total_mean_enum,pri_funding_subsidy_mean_enum,count_register_year_2017_mean,child_observe_total_mean,child_gender,id_facility_mean,id_facility,count_staff_salary_unpaid,child_zha,teacher_emotional_understand,id_mn_n_std_enum,count_register_year_2020,count_register_year_2017_mean_enum,child_observe_interested


In [69]:
ss.to_csv('first_elom.csv',index=None)